In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import pickle
import pyarrow
import fastparquet

In [2]:
# vars
path = "C:/Users/LL/Documents/voter_analysis/merging_data/merged_voter_data_20_24.csv"
# file_name = '100000_voters_sample.csv'

In [3]:
# read
df = pd.read_csv(path)

C:\Users\LL\AppData\Local\Temp\ipykernel_13616\2401616054.py:2: DtypeWarning: Columns (3,4,5,6,8,9,11,14,17,18,22,27,34,35,36,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [4]:
len(df)

33444925

In [5]:
# # a row indicating which year the data is from

# # 2020: 20179326
# # 2024: 13265599

# df['year'] = None
# df.loc[:20179327,'year'] = '2020'
# df.loc[20179327:,'year'] = '2024'

### check unique values

In [6]:
# df['gender'].value_counts()

# where are the null values from - should be 2020 data & voters who didn't vote
# assign U?

In [7]:
# df['enrollment'].value_counts()

In [8]:
# df['otherparty'].value_counts()

In [9]:
# df['vrsource'].value_counts()

# what is OVR

In [10]:
# df['status'].value_counts()

# ACTIVE - A
# PURGED - P
# PREREG - 17
# INACTIVE - I

In [11]:
# df['reasoncode'].value_counts()


In [12]:
# df['rcity'].value_counts()

In [13]:
# df['countycode'].value_counts()

In [14]:
# df['idrequired'].value_counts()

In [ ]:
# df['idmet'].value_counts()



### cleaning

In [16]:
columns = [
            'sboeid', 'dob', 'gender', 'enrollment', 'otherparty', 'regdate', 'vrsource',
           'status', 'reasoncode', 'inact_date', 'purge_date',
            # 'rhalfcode', 'rpredirection', 'rstreetname', 'rpostdirection', 'rapartmenttype', 'rapartment', 'raddrnonstd', 
            # 'rcity', 'rzip5', 'rzip4', 
            # 'mailadd1', 
            # 'countycode', 'ed', 'ld', 'towncity', 'ward', 'cd', 'sd', 'ad', 
            'lastvoteddate', 
            # 'prevyearvoted', 
            # 'prevcounty', 'prevaddress', 'prevname', 
            # 'countyvrnumber', 
            'idrequired', 'idmet', 
            'voterhistory',
            ]
df_cleaning = df[columns]

In [17]:
df_cleaning.head()

,sboeid,dob,gender,enrollment,otherparty,regdate,vrsource,status,reasoncode,inact_date,purge_date,lastvoteddate,idrequired,idmet,voterhistory
0,NY000000000003306194,19710331.0,F,DEM,NaN,19890413.0,CBOE,PURGED,MOVED,NaN,20140611.0,20121106.0,N,Y,2012 General Election;2010 General Election;20...
1,NY000000000003306195,19720601.0,M,REP,NaN,19900607.0,CBOE,ACTIVE,NaN,NaN,NaN,NaN,N,Y,NaN
2,NY000000000003306196,19720721.0,M,DEM,NaN,19910115.0,DMV,ACTIVE,NaN,NaN,NaN,NaN,N,Y,NaN
3,NY000000000003306197,19561106.0,M,REP,NaN,19930226.0,MAIL,INACTIVE,MAILCHECK,20150805.0,NaN,20121106.0,N,Y,2012 General Election;2012 Presidential Primar...
4,NY000000000003306198,19751125.0,F,DEM,NaN,19931202.0,CBOE,ACTIVE,NaN,NaN,NaN,20201103.0,N,Y,2020 General Election;2019 General Election;20...


In [18]:
others = [
            # 'sboeid', 'dob', 'gender', 'enrollment', 'otherparty', 'regdate', 'vrsource',
            # 'status', 'reasoncode', 'inact_date', 'purge_date',
            'rhalfcode', 'rpredirection', 'rstreetname', 'rpostdirection', 'rapartmenttype', 'rapartment', 'raddrnonstd', 
            'rcity', 'rzip5', 'rzip4', 
            'mailadd1', 
            'countycode', 'ed', 'ld', 'towncity', 'ward', 'cd', 'sd', 'ad', 
            # 'lastvoteddate', 
            'prevyearvoted', 
            'prevcounty', 'prevaddress', 'prevname', 
            'countyvrnumber', 
            # 'idrequired', 'idmet', 
            # 'voterhistory'
            ]
df_others = df[others]

#### remove all nan rows

In [19]:
# remove all NaN rows

df_cleaning = df_cleaning.dropna(how='all')

#### voters removed

In [89]:
voters_removed = set()

#### catch out of bound dates then drop the rows

In [20]:
date_columns = ['dob', 'regdate', 'inact_date', 'purge_date', 'lastvoteddate']
df_cleaning[date_columns] = df_cleaning[date_columns].astype('Int64')

In [ ]:
# catch out of bounds index
# (1677, 2262)
max = 22620000
min = 16780000
out_of_bound_idx = list()

out_of_bound_idx.append(df_cleaning.loc[(df_cleaning['dob'] >= max) | (df_cleaning['dob'] <= min)].index.tolist())
out_of_bound_idx.append(df_cleaning.loc[(df_cleaning['regdate'] >= max) | (df_cleaning['regdate'] <= min)].index.tolist())
out_of_bound_idx.append(df_cleaning.loc[(df_cleaning['inact_date'] >= max) | (df_cleaning['inact_date'] <= min)].index.tolist())
out_of_bound_idx.append(df_cleaning.loc[(df_cleaning['purge_date'] >= max) | (df_cleaning['purge_date'] <= min)].index.tolist())
out_of_bound_idx.append(df_cleaning.loc[(df_cleaning['lastvoteddate'] >= max) | (df_cleaning['lastvoteddate'] <= min)].index.tolist())

out_of_bound_idx = sorted(out_of_bound_idx)

In [ ]:
# pickle index

set_of_elements = {item for inner_list in out_of_bound_idx for item in inner_list}
oob_index = sorted(list(set_of_elements))
print(len(oob_index))

oo_bound_idx = pd.Index(oob_index)

# with open('oo_bound_idx.pkl', 'wb') as f:
#     pickle.dump(oo_bound_idx, f)

61


In [ ]:
# examine data

df_oob = df_cleaning.loc[oo_bound_idx]
df_oob

In [ ]:
# catch ids

voters_removed.update(df_oob['sboeid'])

remove for data type transformation

In [25]:
remaining = df_cleaning.index.difference(oo_bound_idx)

# Use the remaining index to select rows
df_cleaning = df_cleaning.loc[remaining]

print(len(df_cleaning))

33444862


In [26]:
33444862+61
# correct

33444923

#### convert to datetime64

In [28]:
df_cleaning['dob'] = pd.to_datetime(df_cleaning['dob'].astype(str), format='%Y%m%d')
df_cleaning['regdate'] = pd.to_datetime(df_cleaning['regdate'].astype(str), format='%Y%m%d')

In [29]:
df_cleaning['inact_date'] = pd.to_datetime(df_cleaning['inact_date'].astype(str), format='%Y%m%d', errors = 'coerce')
df_cleaning['purge_date'] = pd.to_datetime(df_cleaning['purge_date'].astype(str), format='%Y%m%d', errors = 'coerce')
df_cleaning['lastvoteddate'] = pd.to_datetime(df_cleaning['lastvoteddate'].astype(str), format='%Y%m%d', errors = 'coerce')

#### other columns
idrequired/idmet  
enrollment/otherparties  
status  
reasoncode

In [30]:
# idrequired/idmet to binary

df_cleaning.loc[:,'idrequired'] = df_cleaning.loc[:,'idrequired'].apply(lambda x: 1 if x == 'Y' else 0 if x == 'N' else np.nan)
df_cleaning.loc[:,'idmet'] = df_cleaning.loc[:,'idmet'].apply(lambda x: 1 if x == 'Y' else 0 if x == 'N' else np.nan)

In [31]:
# upper case

df_cleaning.loc[:,'enrollment'] = df_cleaning.loc[:,'enrollment'].str.upper()
df_cleaning.loc[:,'otherparty'] = df_cleaning.loc[:,'otherparty'].str.upper()

In [ ]:
# status

status_map = {'ACTIVE': 'A', 'PURGED': 'P', 'PREREG': '17', 'INACTIVE': 'I'}
df_cleaning.loc[:,'status'] = df_cleaning.loc[:,'status'].replace(status_map)

In [33]:
# reason code

df_cleaning.loc[:,'reasoncode'] = df_cleaning.loc[:,'reasoncode'].replace('MAIL-CHECK', 'MAILCHECK')

#### gender

In [34]:
# check null sources in gender

import pickle

gender_blank_idx = df_cleaning.loc[(df_cleaning['gender'] == ' ')].index

# # Save index to a file
# with open('gender_blank_idx.pkl', 'wb') as f:
#     pickle.dump(gender_blank_idx, f)


In [93]:
len(df.loc[gender_blank_idx, 'sboeid'].unique())

250

In [94]:
voters_removed.update(df.loc[gender_blank_idx, 'sboeid'])
print(len(voters_removed))

296


In [35]:
remaining_ = df_cleaning.index.difference(gender_blank_idx)
df_cleaning = df_cleaning.loc[remaining_]

In [36]:
33444862-250
# ccorrect

33444612

#### year_diff column

In [ ]:
df_cleaning['year_diff'] = (df_cleaning['regdate'] - df_cleaning['dob']).dt.days / 365.25

#### merge enrollment and otherparty

In [64]:
# df_cleaning['enrollment'].fillna(df_cleaning['otherparty']).isna().sum() - 0
df_temp = pd.DataFrame()

df_temp['enrollment_combined'] = df_cleaning['enrollment'].fillna(df_cleaning['otherparty'])
df_temp['enrollment_combined']


0           DEM
1           REP
2           DEM
3           REP
4           DEM
           ... 
33444920    BLK
33444921    WOR
33444922    DEM
33444923    BLK
33444924    BLK
Name: enrollment_combined, Length: 33444612, dtype: object

In [65]:
df_cleaning.drop(columns = ['enrollment', 'otherparty'], inplace = True)
df_cleaning['enrollment'] = df_temp['enrollment_combined']

#### complete dataset

In [66]:
result = df_cleaning.join(df_others, how='left')

In [67]:
print(len(result) == 33444612)
print(df_cleaning['regdate'].min())

True
1820-05-18 00:00:00


#### age

too old

In [ ]:
limit = pd.to_datetime('1900-01-01')
df_tooold = df_cleaning.loc[(df_cleaning['dob'] < limit) == True]

In [ ]:
# examine data
df_tooold['year_diff'].max()

197.60164271047228

In [ ]:
# pickle index
too_old_indx = df_tooold.index

with open('too_old_indx.pkl', 'wb') as f:
    pickle.dump(too_old_indx, f)

In [ ]:
# catch ids
voters_removed.update(df_tooold['sboeid'])
print(len(voters_removed))

20623


too young

In [ ]:
# examine data
df_cleaning.loc[df_cleaning['year_diff'] < 16]

In [ ]:
# catch ids
voters_removed.update(df_cleaning.loc[df_cleaning['year_diff'] < 16,'sboeid'])
print(len(voters_removed))

31033


 remove together

In [ ]:
result = result.loc[(result['sboeid'].isin(voters_removed)) == False]
print(len(result))

33405005


#### what we removed so far
count the number of voters we should remove:  <br>
    1. out of bound dates 61<br>
    2. gender blank 250 <br>
    3. too old (typo) 22844 <br>
    4. too young when register (typo) <br>
    total: 31033 <br>
    <br>

#### output

In [103]:
# output

file_name = 'prog_20_24.csv'
result.to_csv(file_name, index=False)

In [3]:
result = pd.read_csv('C:/Users/LL/Documents/voter_analysis/prog_20_24.csv')
result.head()

# 23 min

C:\Users\LL\AppData\Local\Temp\ipykernel_6564\590179374.py:1: DtypeWarning: Columns (7,8,9,12,15,16,18,19,21,24,30,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  result = pd.read_csv('C:/Users/LL/Documents/voter_analysis/prog_20_24.csv')


,sboeid,dob,gender,regdate,vrsource,status,reasoncode,inact_date,purge_date,lastvoteddate,...,towncity,ward,cd,sd,ad,prevyearvoted,prevcounty,prevaddress,prevname,countyvrnumber
0,NY000000000003306194,1971-03-31,F,1989-04-13,CBOE,P,MOVED,NaN,2014-06-11,2012-11-06,...,N Tonawanda,3.0,26.0,62.0,140.0,NaN,,NaN,NaN,M098132
1,NY000000000003306195,1972-06-01,M,1990-06-07,CBOE,A,NaN,NaN,NaN,NaN,...,Town of Lockport,0.0,27.0,62.0,144.0,NaN,,NaN,NaN,M105403
2,NY000000000003306196,1972-07-21,M,1991-01-15,DMV,A,NaN,NaN,NaN,NaN,...,Town of Niagara,0.0,27.0,62.0,145.0,NaN,,NaN,NaN,M109185
3,NY000000000003306197,1956-11-06,M,1993-02-26,MAIL,I,MAILCHECK,2015-08-05,NaN,2012-11-06,...,Somerset,0.0,27.0,62.0,144.0,NaN,,NaN,NaN,M136177
4,NY000000000003306198,1975-11-25,F,1993-12-02,CBOE,A,NaN,NaN,NaN,2020-11-03,...,Pendleton,0.0,27.0,62.0,146.0,NaN,,NaN,NaN,M142110


#### generating switching behavior

In [67]:
columns = ['sboeid', 'enrollment', 'regdate']
df_tmp = result[columns]
df_tmp

,sboeid,enrollment,regdate
0,NY000000000003306194,DEM,1989-04-13
1,NY000000000003306195,REP,1990-06-07
2,NY000000000003306196,DEM,1991-01-15
3,NY000000000003306197,REP,1993-02-26
4,NY000000000003306198,DEM,1993-12-02
...,...,...,...
33405000,NY000000000063652443,BLK,2024-05-23
33405001,NY000000000063652444,WOR,2024-05-22
33405002,NY000000000063652445,DEM,2024-05-22
33405003,NY000000000063652447,BLK,2024-05-23


In [66]:
gby = df_tmp.groupby(by='sboeid')
len(gby)

20312484

In [6]:
print(gby.size().max())
print(gby.size().idxmax())

12
NY000000000021973253


draw 1000 sample groups for execution time estimation

In [75]:
# Get the first 10000 frames (groups)
first_ten_frames = pd.DataFrame()

for i, (_, group) in enumerate(gby):
    if i < 10000:
        first_ten_frames = pd.concat([first_ten_frames, group])
    else:
        break

first_ten_frames

,sboeid,enrollment,regdate
1508804,NY000000000003306104,WOR,2007-05-16
20980570,NY000000000003306104,WOR,2007-05-16
1508805,NY000000000003306105,REP,2006-10-26
14284189,NY000000000003306105,REP,2012-09-07
27544462,NY000000000003306105,REP,2012-09-07
...,...,...,...
20151901,NY000000000008510545,REP,1999-09-30
8028,NY000000000008510546,REP,2004-04-20
20151902,NY000000000008510546,REP,2004-04-20
8029,NY000000000008510547,BLK,2001-12-13


In [76]:
# group by obj
test_gby = first_ten_frames.groupby('sboeid')

# new columns
sample = pd.DataFrame()
sample['switched'] = test_gby['enrollment'].apply(lambda x: x != x.shift()).astype(int)
sample.reset_index(inplace=True, level = 'sboeid', drop = True)
sample['from'] = test_gby['enrollment'].shift()

# when from is nan, assign switched to 0, when switched is 0, assign from to nan
sample.loc[(sample['from'].isna() == True),'switched'] = 0
sample.loc[(sample['switched'] == 0),'from'] = np.nan

sample

,switched,from
1508804,0,NaN
20980570,0,NaN
1508805,0,NaN
14284189,0,NaN
27544462,0,NaN
...,...,...
20151901,0,NaN
8028,0,NaN
20151902,0,NaN
8029,0,NaN


In [77]:
# merge two dataset on index
new = first_ten_frames.join(sample, how = 'left')
print(len(new) == len(sample))
print(new)

True
                        sboeid enrollment     regdate  switched from
1508804   NY000000000003306104        WOR  2007-05-16         0  NaN
20980570  NY000000000003306104        WOR  2007-05-16         0  NaN
1508805   NY000000000003306105        REP  2006-10-26         0  NaN
14284189  NY000000000003306105        REP  2012-09-07         0  NaN
27544462  NY000000000003306105        REP  2012-09-07         0  NaN
...                        ...        ...         ...       ...  ...
20151901  NY000000000008510545        REP  1999-09-30         0  NaN
8028      NY000000000008510546        REP  2004-04-20         0  NaN
20151902  NY000000000008510546        REP  2004-04-20         0  NaN
8029      NY000000000008510547        BLK  2001-12-13         0  NaN
8030      NY000000000008510548        BLK  2001-12-13         0  NaN

[17083 rows x 5 columns]


1.4s * 1956 / 60 ~ 45 min

the whole dataset

In [78]:
# new columns
new_col = pd.DataFrame()
new_col['switched'] = gby['enrollment'].apply(lambda x: x != x.shift()).astype(int)
new_col.reset_index(inplace=True, level = 'sboeid', drop = True)
new_col['from'] = gby['enrollment'].shift()

# when from is nan, assign switched to 0, when switched is 0, assign from to nan
new_col.loc[(new_col['from'].isna() == True),'switched'] = 0
new_col.loc[(new_col['switched'] == 0),'from'] = np.nan

new_col.head()

,switched,from
1508804,0,NaN
20980570,0,NaN
1508805,0,NaN
14284189,0,NaN
27544462,0,NaN


In [79]:
# merge two dataset on index
new_result= result.join(new_col, how = 'left')
print(len(new_result) == len(new_col))
print(new_result.head())

True
                 sboeid         dob gender     regdate vrsource status  \
0  NY000000000003306194  1971-03-31      F  1989-04-13     CBOE      P   
1  NY000000000003306195  1972-06-01      M  1990-06-07     CBOE      A   
2  NY000000000003306196  1972-07-21      M  1991-01-15      DMV      A   
3  NY000000000003306197  1956-11-06      M  1993-02-26     MAIL      I   
4  NY000000000003306198  1975-11-25      F  1993-12-02     CBOE      A   

  reasoncode  inact_date  purge_date lastvoteddate  ...    cd    sd     ad  \
0      MOVED         NaN  2014-06-11    2012-11-06  ...  26.0  62.0  140.0   
1        NaN         NaN         NaN           NaN  ...  27.0  62.0  144.0   
2        NaN         NaN         NaN           NaN  ...  27.0  62.0  145.0   
3  MAILCHECK  2015-08-05         NaN    2012-11-06  ...  27.0  62.0  144.0   
4        NaN         NaN         NaN    2020-11-03  ...  27.0  62.0  146.0   

   prevyearvoted prevcounty prevaddress prevname countyvrnumber switched from  
0

In [80]:
file_name = 'progress_3_20_24.csv'
new_result.to_csv(file_name, index=False)

In [ ]:
# check the result

selected_col = ['sboeid','enrollment', 'from', 'switched']
# new_result.loc[new_result['switched'] == 1, selected_col].head()
new_result.loc[new_result['sboeid'] == 'NY000000000008507914', selected_col]

,sboeid,enrollment,from,switched
5396,NY000000000008507914,IND,NaN,0
313441,NY000000000008507914,DEM,IND,1


### problems

#### loading

In [ ]:
# import

new_result = pd.read_csv('C:/Users/LL/Documents/voter_analysis/progress_3_20_24.csv',
                         dtype = {'gender': 'category', 'from': 'category', 'enrollment': 'category', 'vrsource': 'category', 'status': 'category', 'reasoncode': 'category', 'countycode': 'category', 'idrequired': 'category', 'idmet': 'category'})

C:\Users\LL\AppData\Local\Temp\ipykernel_16312\484325796.py:3: DtypeWarning: Columns (7,8,9,12,15,16,18,19,21,24,30,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  new_result = pd.read_csv('C:/Users/LL/Documents/voter_analysis/progress_3_20_24.csv',


In [ ]:
selected_col = ['sboeid','enrollment', 'from', 'switched']

new_result.loc[:10, selected_col]

,sboeid,enrollment,from,switched
0,NY000000000003306194,DEM,NaN,0
1,NY000000000003306195,REP,NaN,0
2,NY000000000003306196,DEM,NaN,0
3,NY000000000003306197,REP,NaN,0
4,NY000000000003306198,DEM,NaN,0
5,NY000000000003306199,BLK,NaN,0
6,NY000000000003306200,DEM,NaN,0
7,NY000000000003306201,IND,NaN,0
8,NY000000000003306202,CON,NaN,0
9,NY000000000003306203,DEM,NaN,0



if we remove one record from those people we should remove everything: <br>

bad register date?? <br>
verify because we need switching_date <br>
but how, year_diff is affected by both dob and regdate <br>

In [ ]:
# new_result['dob'] = pd.to_datetime(new_result['dob'])
# new_result['regdate'] = pd.to_datetime(new_result['regdate'])

#### purged 
remove reasoncode: COURT, DUPLICATE <br>
INACTIVE: can be purged when stay inactive for too long, doesn't matter when examining switching behavior as long as they are allow to enter later.

remove DUPLICATE 7 voters

In [112]:
dup_ids = set()
dup_ids.update(new_result.loc[(new_result['reasoncode'] == 'DUPLICATE'), 'sboeid'])
# len(dup_ids) # 4

new_result = new_result[~(new_result['sboeid'].isin(dup_ids))]

remove COURT 69652 voters

In [113]:
# reasoncode instead
# COURT: 67354
# COURT & P: 67354

# len(new_result.loc[(new_result['reasoncode'] == 'COURT') & (new_result['status'] == 'P')])

court_ids = set()
court_ids.update(new_result.loc[(new_result['reasoncode'] == 'COURT'), 'sboeid'])
# len(court_ids) # 67328

new_result = new_result[~(new_result['sboeid'].isin(court_ids))]

#### remove year_diff more than 120 8 voters

In [115]:
year_diff_toobig_ids = set()
year_diff_toobig_ids.update(new_result.loc[(new_result['year_diff'] > 120), 'sboeid'])
print(len(year_diff_toobig_ids))

new_result = new_result[~(new_result['sboeid'].isin(year_diff_toobig_ids))]

8


In [119]:
# change type to int8
new_result['year_diff'] = new_result['year_diff'].astype('int8')

#### more examining

 status = 17 <br>
 pre-register: 141365 <br>
pre-register & enrolled: 74740 <br>
pre-reg people: 141358 <br>

-------------------------------------------- <br>
system flaws: <br>
those who got updated have VERY inconsistent records: hard to determine if they actively registered again or no so we assume: not automatically active upon turning 18 <br>
they can vote while being '17', which means the system don't autometically update the record when they turn 18, who would build a system like this?? isn't this the whole purpose of PRE register? <br>

human error (that could have been avoided by having a more robust system): <br>
got assigned the wrong status code, too old to be '17' <br>

also, some of the register dates are the same when they became active and some are not? moreover, WHO built this system?? <br>
somebody's birthday changed?? dude don't you put restraint on your database??? ever heard of data integrity?? <br>

it means I can't trust the DE at all and I have to verify everything

In [79]:
# print(len(new_result.loc[(new_result['status'] == '17')])) # pre-reg

# print(new_result.loc[(new_result['status'] == '17') & (new_result['enrollment'] != 'BLK')])  # pre-reg & enrolled

prereg_ids = set()
prereg_ids.update(new_result.loc[(new_result['status'] == '17'), 'sboeid'])
len(prereg_ids)

141358

------------------------------------------------------ <br>
potentially corrupted columns (that we can tell): (descending in terms of importance)<br>

dob (i dont believe it) <br>
regdate <br>
status <br>
reasoncode <br>
lastvoteddate ~ verify against voterhistory <br>

inact_date, purge_date <br>

---- <br>
more cleaning plan: <br>

lastvotedyear is not necessary and are mostly just nan <br>
lastvoteddate can be verified against voterhistory <br>

---- <br>
Also what is up with the dictionary? There is no way to tell under what condition they update data in what way <br>

##### loc verify

reason code doesn't match status (purged for inactive reasons) <br>

In [98]:
new_result['reasoncode'].value_counts()

reasoncode
OTHER          2712364
MOVED          2322749
DEATH          1448169
MAILCHECK      1376590
NCOA            465907
INACTIVE        275678
RETURN-MAIL     260798
COURT            67354
DUPLICATE            4
Name: count, dtype: int64

In [ ]:
# all records with reasoncode belongs to inactive have inactive status GOOD
reasons_inactive = ['MAILCHECK', 'RETURN-MAIL', 'NCOA']
len(new_result.loc[(new_result['reasoncode'].isin(reasons_inactive)) & (new_result['status'] != 'I')])

0

In [100]:
new_result['status'].value_counts()

status
A     24288676
P      6550640
I      2378973
17      141365
AF       32675
AU       10330
AM        2344
AP           2
Name: count, dtype: int64

In [ ]:
# no record that has reasoncode present for purging but not inactive or purged GOOD
reasons_present = ['OTHER', 'MOVED', 'DEATH', 'INACTIVE', 'COURT', 'DUPLICATE', 'MAILCHECK', 'NCOA', 'RETURN-MAIL']
status_active = ['A', '17', 'AF', 'AU', 'AM', 'AP']
len(new_result.loc[(new_result['reasoncode'].isin(reasons_present)) 
                   & (new_result['status'].isin(status_active)) 
                   & (new_result['status'].isna() == False)])

0

assigned wrong status - status: 17 when too old 8 voters <br>

In [130]:
examining_col = ['dob', 'regdate', 'year_diff', 'status']
len(new_result.loc[(new_result['status'] == '17') & (new_result['year_diff'] > 18), examining_col])

# generating voter ids
not_actually_17_ids = set()
not_actually_17_ids.update(new_result.loc[(new_result['status'] == '17') & (new_result['year_diff'] > 18), 'sboeid'])

In [131]:
new_result = new_result[~(new_result['sboeid'].isin(not_actually_17_ids))]

##### groupby verify

wrong reason code - death then come back  253 voters <br>

In [ ]:
deseased_ids = set()
deseased_ids.update(new_result.loc[(new_result['reasoncode'] == 'DEATH'), 'sboeid'])

deseased_df = new_result.loc[(new_result['sboeid'].isin(deseased_ids))].sort_values('regdate')

last_reason = deseased_df.groupby('sboeid')['reasoncode'].last().reset_index()

In [ ]:
# it's sorted
deseased_df.loc[deseased_df['sboeid'] == 'NY000000000009254279']

,sboeid,dob,gender,regdate,vrsource,status,reasoncode,inact_date,purge_date,lastvoteddate,...,cd,sd,ad,prevyearvoted,prevcounty,prevaddress,prevname,countyvrnumber,switched,from
5155346,NY000000000009254279,1946-08-22,M,1994-10-14,CBOE,P,DEATH,NaN,2010-11-19,NaN,...,25.0,55.0,137.0,NaN,,NaN,NaN,008902245,0,NaN
180956,NY000000000009254279,1946-08-22,M,2000-09-25,CBOE,P,MOVED,NaN,2007-08-08,NaN,...,24.0,50.0,129.0,NaN,,NaN,NaN,J41918,0,NaN


In [ ]:
# one example
new_result.loc[(new_result['sboeid'] == 'NY000000000009254279')]

,sboeid,dob,gender,regdate,vrsource,status,reasoncode,inact_date,purge_date,lastvoteddate,...,cd,sd,ad,prevyearvoted,prevcounty,prevaddress,prevname,countyvrnumber,switched,from
180956,NY000000000009254279,1946-08-22,M,2000-09-25,CBOE,P,MOVED,NaN,2007-08-08,NaN,...,24.0,50.0,129.0,NaN,,NaN,NaN,J41918,0,NaN
5155346,NY000000000009254279,1946-08-22,M,1994-10-14,CBOE,P,DEATH,NaN,2010-11-19,NaN,...,25.0,55.0,137.0,NaN,,NaN,NaN,008902245,0,NaN


In [190]:
vampires = set()
vampires.update(last_reason.loc[(last_reason['reasoncode'] != 'DEATH'), 'sboeid'])
len(vampires)

253

In [191]:
new_result = new_result[~(new_result['sboeid'].isin(vampires))]

date of birth changed (remove all voters with corrupted data or try to recover) 21375 voters<br>
didn't remove, pickled all the ids

In [ ]:
unique_dob = new_result.groupby('sboeid')['dob'].nunique()
dob_counts_df = unique_dob.reset_index()

In [ ]:
len(dob_counts_df.loc[(dob_counts_df['dob'] != 1)])

bad_dob_id = set()
bad_dob_id.update(dob_counts_df.loc[(dob_counts_df['dob'] != 1), 'sboeid'])

# new_result = new_result[~(new_result['sboeid'].isin(bad_dob_id))]
len(bad_dob_id)

21375

In [222]:
# pickle the object

with open('bad_dob_id.pkl', 'wb') as f:
    pickle.dump(bad_dob_id, f)

repeating register dates across a few records of one voter - especially when present in 'status' == '17' <br>

In [204]:
unique_d_counts = new_result.groupby('sboeid')['regdate'].nunique()
group_sizes = new_result.groupby('sboeid').size()

In [207]:
counts_df = unique_d_counts.reset_index()
sizes_df = group_sizes.reset_index()

In [208]:
regdate_repeat = pd.merge(counts_df, sizes_df, on='sboeid', how='left')
print(len(regdate_repeat) == len(sizes_df))

True


In [ ]:
regdate_repeat.loc[(regdate_repeat['regdate'] != regdate_repeat[0])]
# 11437367
# too many, can't do

,sboeid,regdate,0
0,NY000000000003306104,1,2
1,NY000000000003306105,2,3
3,NY000000000003306107,1,2
4,NY000000000003306108,2,3
5,NY000000000003306109,1,2
...,...,...,...
18938081,NY000000000060075856,1,2
18938082,NY000000000060075857,1,2
18938083,NY000000000060075858,1,2
18938084,NY000000000060075859,1,2


### save progress 4th

In [ ]:
progress4_idx = new_result.index

with open('progress4_idx.pkl', 'wb') as f:
    pickle.dump(progress4_idx, f)

### loading
i don't believe it vs code you crushed again?

In [3]:
with open('progress4_idx.pkl', 'rb') as file:
    index = pickle.load(file)

In [4]:
# import

new_result = pd.read_csv('C:/Users/LL/Documents/voter_analysis/progress_3_20_24.csv',
                         dtype = {'gender': 'category', 'from': 'category', 'enrollment': 'category', 'vrsource': 'category', 'status': 'category', 'reasoncode': 'category', 'countycode': 'category', 'idrequired': 'category', 'idmet': 'category'})

C:\Users\LL\AppData\Local\Temp\ipykernel_11740\484325796.py:3: DtypeWarning: Columns (7,8,9,12,15,16,18,19,21,24,30,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  new_result = pd.read_csv('C:/Users/LL/Documents/voter_analysis/progress_3_20_24.csv',


In [8]:
new_result = new_result.loc[new_result.index.isin(index)]

In [11]:
len(index) == len(new_result)

True

### parse dob, regdate, lastdatevoted

In [15]:
dtype_mapping = {
    'dob': 'datetime64[ns]',
    'regdate': 'datetime64[ns]',
    'lastvoteddate': 'datetime64[ns]',
}

# Use astype() to change datatypes based on the dictionary
new_result = new_result.astype(dtype_mapping)

In [17]:
new_result['dob_year'] = new_result['dob'].dt.year.astype('category')

new_result['lastvoteddate_year'] = new_result['lastvoteddate'].dt.year.astype('category')
new_result['lastvoteddate_month'] = new_result['lastvoteddate'].dt.month.astype('category')
new_result['lastvoteddate_day'] = new_result['lastvoteddate'].dt.day.astype('category')
new_result['lastvoteddate_dayofweek'] = new_result['lastvoteddate'].dt.dayofweek.astype('category')  # Monday=0, Sunday=6

new_result['regdate_year'] = new_result['regdate'].dt.year.astype('category')
new_result['regdate_month'] = new_result['regdate'].dt.month.astype('category')
new_result['regdate_day'] = new_result['regdate'].dt.day.astype('category')
new_result['regdate_dayofweek'] = new_result['regdate'].dt.dayofweek.astype('category') # Monday=0, Sunday=6

### compress data

inspect mixed datatype

In [26]:
columns_inspecting = ['inact_date','purge_date', 'voterhistory', 'year_diff', 'rhalfcode', 'rpredirection', 'rstreetname', 'rpostdirection', 'rapartmenttype', 'rapartment', 'raddrnonstd', 'rcity', 'rzip4', 'mailadd1', 'ed', 'ld', 'towncity', 'ward', 'cd', 'sd', 'ad', 'prevyearvoted', 'prevcounty', 'prevaddress', 'prevname', 'countyvrnumber']

list_of_types = []

for col in columns_inspecting:

    types = new_result[col].apply(type)

    list_of_types.append(types.value_counts())

sboeid                             | object - str <br>

inact_date                         | object - float, str <br>
purge_date                         | object - float, str <br>

voterhistory                       | object - float, str <br>

rhalfcode                          | object f s <br>
rpredirection                      | object f s <br>
rstreetname                        | object f s <br>
rpostdirection                     | object f s <br>
rapartmenttype                     | object f s <br>
rapartment                         | object f s <br>
raddrnonstd                        | object f s <br>
rcity                              | object f s <br>

rzip4                              | object f s <br>
mailadd1                           | object f s <br>

towncity                           | object f s <br>
ward                               | object - float, str, int <br>

prevcounty                         | object - string <br>
prevaddress                        | object f s <br>
prevname                           | object f s <br>
countyvrnumber                     | object - str, int <br>

potentially categorical: <br>
    'ed', 
    'ld', 
    'cd', 
    'sd', 
    'ad', <br>
    'rcity', 
    'ward', 
    'prevcounty',
    'rzip4',
    

In [ ]:
# (new_result['rzip4'].unique())

array([1948.0, nan, 1936.0, ..., 781.0, 8198.0, 7798.0], dtype=object)

##### compressing plan

In [84]:
# drop 'prevyearvoted': empty column
new_result = new_result.drop(columns = ['prevyearvoted'])

problem: do I just assume '1 ' is 01 or no, probably no

In [ ]:
new_result.loc[new_result['prevcounty'] == '1 ', 'prevcounty'] = ' '  # make sure the algo convert it to nan

In [ ]:
# prevcounty convert to int then sort, where were those extra 4 county codes from??? ('  ', '1 ', '99', '63') to int64 then cate

new_result['prevcounty'] = pd.to_numeric(new_result['prevcounty'], errors='coerce').astype("Int64")

In [ ]:
# zipcode plus 4 has strings?? HUh

# type_zip4 = new_result['rzip4'].apply(type).astype(str)
# wtf_zip4_index = type_zip4[type_zip4 == "<class 'str'>"].index
# new_result.loc[new_result.index.isin(wtf_zip4_index),'rzip4'] 

# WHY PANDAS WHY those are strings but look like floats

zip4_numeric = pd.to_numeric(new_result['rzip4'], errors="coerce").astype("Int64")
print(zip4_numeric.isna().sum())
print(new_result['rzip4'].isna().sum())
new_result['rzip4'] = zip4_numeric

20892057
20892045


In [171]:
# float remove decimal point and one digit after decimal, convert to categorical data
# 'ed', 'ld', 'cd', 'sd', 'ad'
new_result['ed'] = new_result['ed'].astype("Int64")
new_result['ld'] = new_result['ld'].astype("Int64")
new_result['cd'] = new_result['cd'].astype("Int64")
new_result['sd'] = new_result['sd'].astype("Int64")
new_result['ad'] = new_result['ad'].astype("Int64")

In [179]:
# 'rzip5'
# I don't even know cuz the length of the zipcode is not consistent, just please fire the dude who wrote this system right now

# len() == 7 remove the last two chracters
# len() == 6 remove the decimal
# len() == 4 idek whatever just remove the decimal

zip5 = new_result['rzip5'].astype('string').apply(lambda x: x.replace('.', '') if pd.notna(x) else x).astype('string').apply(lambda x: x[:-1] if pd.notna(x) and len(x) == 6 else x)


In [ ]:
new_result = new_result.drop(columns = ['rzip5'])
new_result['rzip5'] = zip5.astype('string')

In [172]:
# just convert to string then to cate
# 'rcity', 'ward'
new_result['rcity'] = new_result['rcity'].astype('string')
new_result['ward'] = new_result['ward'].astype('string')

In [187]:
# prepare for parquet format

dtype_mapping_2 = {
    # booleans          
    'switched': 'bool',
    # categorical
    'rzip5': 'category',
    'rzip4': 'category',
    'rcity': 'category',
    'ed': 'category', 
    'ld': 'category', 
    'cd': 'category', 
    'sd': 'category', 
    'ad': 'category',
    'prevcounty': 'category',
    'rcity': 'category', 
    'ward': 'category', 
    # (nullable) datetime64:
    'inact_date': 'datetime64[ns]',
    'purge_date': 'datetime64[ns]',
    # (nullable) integers

    # (nullable) strings
    'sboeid': 'string',
    'voterhistory': 'string',
    'rhalfcode': 'string', 
    'rpredirection': 'string', 
    'rstreetname': 'string', 
    'rpostdirection': 'string', 
    'rapartmenttype': 'string', 
    'rapartment': 'string', 
    'raddrnonstd': 'string', 
    'mailadd1': 'string', 
    'towncity': 'string', 
    'prevaddress': 'string', 
    'prevname': 'string',
    'countyvrnumber': "string"
    }



# Use astype() to change datatypes based on the dictionary
new_result = new_result.astype(dtype_mapping_2, errors = 'ignore')

In [188]:
new_result.dtypes

sboeid                     string[python]
dob                        datetime64[ns]
gender                           category
regdate                    datetime64[ns]
vrsource                         category
status                           category
reasoncode                       category
inact_date                 datetime64[ns]
purge_date                 datetime64[ns]
lastvoteddate              datetime64[ns]
idrequired                       category
idmet                            category
voterhistory               string[python]
year_diff                         float64
enrollment                       category
rhalfcode                  string[python]
rpredirection              string[python]
rstreetname                string[python]
rpostdirection             string[python]
rapartmenttype             string[python]
rapartment                 string[python]
raddrnonstd                string[python]
rcity                            category
rzip4                            c

In [306]:
new_result.head()

,sboeid,dob,gender,regdate,vrsource,status,reasoncode,inact_date,purge_date,lastvoteddate,...,dob_day,dob_dayofweek,lastvoteddate_year,lastvoteddate_month,lastvoteddate_day,lastvoteddate_dayofweek,regdate_year,regdate_month,regdate_day,regdate_dayofweek
0,NY000000000003306194,1971-03-31,F,1989-04-13,CBOE,P,MOVED,NaT,2014-06-11,2012-11-06,...,31,2,2012.0,11.0,6.0,1.0,1989,4,13,3
1,NY000000000003306195,1972-06-01,M,1990-06-07,CBOE,A,NaN,NaT,NaT,NaT,...,1,3,NaN,NaN,NaN,NaN,1990,6,7,3
2,NY000000000003306196,1972-07-21,M,1991-01-15,DMV,A,NaN,NaT,NaT,NaT,...,21,4,NaN,NaN,NaN,NaN,1991,1,15,1
3,NY000000000003306197,1956-11-06,M,1993-02-26,MAIL,I,MAILCHECK,2015-08-05,NaT,2012-11-06,...,6,1,2012.0,11.0,6.0,1.0,1993,2,26,4
4,NY000000000003306198,1975-11-25,F,1993-12-02,CBOE,A,NaN,NaT,NaT,2020-11-03,...,25,1,2020.0,11.0,3.0,1.0,1993,12,2,3


### export data as .parquet format

In [189]:
new_result.to_parquet('clean.parquet')